In [1]:
import requests
import pandas as pd
from datetime import datetime

In [8]:
# Replace these variables with the owner and repository name of the repo you want to query
url = f"https://github.com/ebmdatalab/openprescribing/tree/main/openprescribing/measures/definitions/"

# Optionally, if you have a GitHub token, you can use it for authenticated requests to increase the rate limit
token = None  # Replace with your GitHub token or set to None if not using one

headers = {"Authorization": f"token {token}"} if token else {}

In [9]:
def fetch_commits(url, headers):
    commits = []
    while url:
        response = requests.get(url, headers=headers)
        
        # Debugging information
        print(f"Fetching: {url}")
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            try:
                data = response.json()
                commits.extend(data)
                # Check if there's a next page
                if 'next' in response.links:
                    url = response.links['next']['url']
                else:
                    url = None
            except json.JSONDecodeError as e:
                print(f"JSON Decode Error: {e}")
                print(f"Response content: {response.text}")
                break
        else:
            if response.status_code == 403:
                print("Rate limit exceeded. Try again later.")
            else:
                print(f"Error fetching commits: {response.status_code}")
                print(f"Response content: {response.text}")
            break
    return commits

commits = fetch_commits(url, headers)

# Extract useful information
commit_data = []
for commit in commits:
    commit_data.append({
        'sha': commit['sha'],
        'author': commit['commit']['author']['name'],
        'message': commit['commit']['message'],
        'date': datetime.strptime(commit['commit']['author']['date'], "%Y-%m-%dT%H:%M:%SZ")
    })

# Create a DataFrame
df = pd.DataFrame(commit_data)

# Display the first few rows of the DataFrame
df.head()

Fetching: https://github.com/ebmdatalab/openprescribing/tree/main/openprescribing/measures/definitions/
Status Code: 200


NameError: name 'json' is not defined

In [7]:
commits = fetch_commits(url, headers)

# Extract useful information
commit_data = []
for commit in commits:
    commit_data.append({
        'sha': commit['sha'],
        'author': commit['commit']['author']['name'],
        'message': commit['commit']['message'],
        'date': datetime.strptime(commit['commit']['author']['date'], "%Y-%m-%dT%H:%M:%SZ")
    })

# Create a DataFrame
df = pd.DataFrame(commit_data)

JSONDecodeError: Expecting value: line 7 column 1 (char 6)

In [18]:
import requests
import pandas as pd
from datetime import datetime
import json

# Replace these variables with the owner and repository name of the repo you want to query
owner = "ebmdatalab"  # e.g., "octocat"
repo = "openprescribing"  # e.g., "Hello-World"
base_url = f"https://api.github.com/repos/{owner}/{repo}/pulls"

# Optionally, if you have a GitHub token, you can use it for authenticated requests to increase the rate limit
token = None  # Replace with your GitHub token or set to None if not using one

headers = {"Authorization": f"token {token}"} if token else {}

def fetch_pull_requests(url, state, headers):
    pull_requests = []
    page = 1
    
    while True:
        response = requests.get(f"{url}?state={state}&page={page}&per_page=100", headers=headers)
        
        # Debugging information
        print(f"Fetching: {url}?state={state}&page={page}&per_page=100")
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            try:
                data = response.json()
                if not data:
                    break
                pull_requests.extend(data)
                page += 1
            except json.JSONDecodeError as e:
                print(f"JSON Decode Error: {e}")
                print(f"Response content: {response.text}")
                break
        else:
            if response.status_code == 403:
                print("Rate limit exceeded. Try again later.")
            else:
                print(f"Error fetching pull requests: {response.status_code}")
                print(f"Response content: {response.text}")
            break
    return pull_requests

all_pull_requests = []

for state in ["open", "closed"]:
    print(f"Fetching {state} pull requests...")
    prs = fetch_pull_requests(base_url, state, headers)
    all_pull_requests.extend(prs)

# Extract useful information
pr_data = []
for pr in all_pull_requests:
    labels = [label['name'] for label in pr['labels']]
    pr_data.append({
        'id': pr['id'],
        'number': pr['number'],
        'title': pr['title'],
        'user': pr['user']['login'],
        'state': pr['state'],
        'created_at': datetime.strptime(pr['created_at'], "%Y-%m-%dT%H:%M:%SZ"),
        'updated_at': datetime.strptime(pr['updated_at'], "%Y-%m-%dT%H:%M:%SZ"),
        'closed_at': datetime.strptime(pr['closed_at'], "%Y-%m-%dT%H:%M:%SZ") if pr['closed_at'] else None,
        'merged_at': datetime.strptime(pr['merged_at'], "%Y-%m-%dT%H:%M:%SZ") if pr['merged_at'] else None,
        'branch': pr['head']['ref'],  # Branch name
        'labels': labels  # List of labels
    })

# Create a DataFrame
df = pd.DataFrame(pr_data)

# Display the first few rows of the DataFrame
df.head()



Fetching open pull requests...
Fetching: https://api.github.com/repos/ebmdatalab/openprescribing/pulls?state=open&page=1&per_page=100
Status Code: 200
Fetching: https://api.github.com/repos/ebmdatalab/openprescribing/pulls?state=open&page=2&per_page=100
Status Code: 200
Fetching closed pull requests...
Fetching: https://api.github.com/repos/ebmdatalab/openprescribing/pulls?state=closed&page=1&per_page=100
Status Code: 200
Fetching: https://api.github.com/repos/ebmdatalab/openprescribing/pulls?state=closed&page=2&per_page=100
Status Code: 200
Fetching: https://api.github.com/repos/ebmdatalab/openprescribing/pulls?state=closed&page=3&per_page=100
Status Code: 200
Fetching: https://api.github.com/repos/ebmdatalab/openprescribing/pulls?state=closed&page=4&per_page=100
Status Code: 200
Fetching: https://api.github.com/repos/ebmdatalab/openprescribing/pulls?state=closed&page=5&per_page=100
Status Code: 200
Fetching: https://api.github.com/repos/ebmdatalab/openprescribing/pulls?state=closed&p

,id,number,title,user,state,created_at,updated_at,closed_at,merged_at,branch,labels
0,1874548984,4793,chore(deps): bump google-cloud-bigquery from 3...,dependabot[bot],open,2024-05-17 00:54:38,2024-05-17 00:54:39,NaT,NaT,dependabot/pip/google-cloud-bigquery-3.23.0,"[dependencies, python]"
1,1874547523,4792,chore(deps): bump selenium from 4.20.0 to 4.21.0,dependabot[bot],open,2024-05-17 00:52:07,2024-05-17 00:52:08,NaT,NaT,dependabot/pip/selenium-4.21.0,"[dependencies, python]"
2,1874545856,4791,chore(deps): bump trio from 0.25.0 to 0.25.1,dependabot[bot],open,2024-05-17 00:49:29,2024-05-17 00:49:30,NaT,NaT,dependabot/pip/trio-0.25.1,"[dependencies, python]"
3,1834510671,4757,Amend measure icsdose,chrisjwood16,open,2024-04-22 12:39:20,2024-04-30 10:42:44,NaT,NaT,amend_measure__icsdose,[]
4,1834400960,4756,Amend measure ktt9 cephalosporins star,chrisjwood16,open,2024-04-22 11:38:33,2024-04-22 11:38:33,NaT,NaT,amend_measure__ktt9_cephalosporins_star,[]


In [25]:
# Filter the DataFrame by branch name and state
filtered_df = df[(df['branch'].str.startswith("new_measure")) & 
                 (df['state'] == 'closed') & 
                 (df['merged_at'].notna())]


# Display the filtered DataFrame
print(filtered_df)

             id  number                                              title  \
196  1699822579    4602  New measure fluoroquinolone items per 1000 reg...   
464  1512199744    4314                        New measure  doxy 100 6 qty   
637  1399512952    4129                             Create pregabalin.json   
688  1360842021    4078                       new_measure__nhse_lancets_v2   
702  1350708151    4063                             new_measure__nhse_bgts   
759  1301563725    4000                 new_measure__amox_500_15_caps.json   

             user   state          created_at          updated_at  \
196  chrisjwood16  closed 2024-01-29 12:02:05 2024-02-20 19:33:13   
464  richiecroker  closed 2023-09-12 14:02:53 2023-09-12 14:28:45   
637  richiecroker  closed 2023-06-20 09:21:53 2023-06-20 09:35:31   
688  richiecroker  closed 2023-05-23 07:43:11 2023-05-23 07:54:10   
702  richiecroker  closed 2023-05-15 12:23:47 2023-05-22 22:18:51   
759  richiecroker  closed 2023-04-04 12

In [26]:
print(df)

              id  number                                              title  \
0     1874548984    4793  chore(deps): bump google-cloud-bigquery from 3...   
1     1874547523    4792   chore(deps): bump selenium from 4.20.0 to 4.21.0   
2     1874545856    4791       chore(deps): bump trio from 0.25.0 to 0.25.1   
3     1834510671    4757                             Amend measure  icsdose   
4     1834400960    4756            Amend measure  ktt9 cephalosporins star   
...          ...     ...                                                ...   
3725    70672114      20                            Make failing tests pass   
3726    70374906      17                                  Dockerise the app   
3727    70190326      14           Update Travis notification email address   
3728    70179374      12         Move the explanatory video below the fold.   
3729    70178339      11               Minor documentation fixes and hints.   

                 user   state          created_at  